In [1]:
import pandas as pd
import numpy as np

In [2]:
#sizes = {'real': {'train': 443757, 'val': 214354}, 'abstract': {'train': 60000, 'val': 30000}}

In [2]:
def get_summary(scenario, _set = 'val'):
    details_train = pd.read_csv('./data/predictions_details_{}_train'.format(scenario), header=None)
    details_train['_set'] = 'train'
    details_val = pd.read_csv('./data/predictions_details_{}_val'.format(scenario), header=None)
    details_val['_set'] = 'val'
    details = pd.concat([details_train, details_val])
    details[2] = details[2].astype(int)
    details.sort_values(by=2, inplace=True)

    prediction = pd.read_csv('./concatenated_data/{}_prediction.csv'.format(scenario))
    prediction.rename(columns={'2049':'q_type', '2048':'q_id'}, inplace=True)
    prediction['base_acc'] = (prediction.base_predicted == prediction.base_target).astype(int)
    prediction['_set'] = details.set_index(2)[['_set']].loc[prediction['q_id'].astype(int)].values
    prediction['my_acc'] = (prediction.my_predicted == prediction.my_target).astype(int)
    prediction['base_acc'] = (prediction.base_predicted == prediction.base_target).astype(int)
    sset_prediction = prediction[prediction._set == _set]
    accs = (sset_prediction[['my_acc','base_acc']].sum()/sset_prediction.shape[0]).values
    print('acc', accs, accs[0] > accs[1])
    accurracies = sset_prediction[['q_type','my_acc','base_acc',
                              'sum_top_down', 'sum_bottom_up']].groupby(['q_type']).sum().reset_index()
    accurracies = accurracies.set_index('q_type')
    
    
    new_summary = pd.read_csv('./concatenated_data/queries_summary_{}.csv'.format(scenario))
    del new_summary['count_top_down']
    new_summary.rename(columns={'2049':'_q_type'}, inplace=True)
    
    
    new_summary = new_summary[new_summary._q_type.isin(accurracies.index.values)]
    new_summary['q_type'] = accurracies.loc[new_summary._q_type.values].index.values
    new_summary['my_acc'] = accurracies.loc[new_summary._q_type.values].my_acc.values
    new_summary['base_acc'] =  accurracies.loc[new_summary._q_type.values].base_acc.values
    val_count = pd.DataFrame(sset_prediction.q_type.value_counts())
    new_summary['_count'] = val_count.loc[new_summary._q_type.values].values
    new_summary['above_median'] = new_summary['_count'] > float(new_summary['_count'].median())
    new_summary['sum_top_down'] = accurracies.loc[new_summary._q_type.values].sum_top_down.values
    new_summary['sum_bottom_up'] =  accurracies.loc[new_summary._q_type.values].sum_bottom_up.values
    new_summary['winner'] = np.array(['top_down',
                                      'bottom_up'])[(new_summary.sum_bottom_up 
                                       > new_summary.sum_top_down).astype(int)]
    _sum = new_summary[['my_acc','base_acc']].sum()
    print('acc_sum', _sum.values, _sum[0] > _sum[1])#/accurracies.shape[0]
    assert (new_summary._q_type != new_summary.q_type).sum() == 0
    _sum = (new_summary.sum_top_down + new_summary.sum_bottom_up).values
    new_summary['sum_top_down'] = new_summary['sum_top_down']/_sum
    new_summary['sum_bottom_up'] = new_summary['sum_bottom_up']/_sum
    new_summary['_diff'] = (new_summary.sum_top_down - new_summary.sum_bottom_up)
    del new_summary['diff']
    del new_summary['q_type']
    del new_summary['sum_top_down']

    print(new_summary._count.sum())
    return new_summary

In [18]:
scenario = 'abstract'
summary = get_summary(scenario)

('acc', array([ 0.6383    ,  0.63913333]), False)
('acc_sum', array([19149, 19174]), False)
30000


In [19]:
#p = 1.31
p = 1.38

mean_dif = summary._diff.mean()
std_dif = summary._diff.std()
data_dif = summary._diff
data_diff = pd.DataFrame( (data_dif < mean_dif - p * std_dif).astype(int) + 
                         (data_dif > mean_dif + p * std_dif).astype(int) > 0 )
significantly_different = summary.loc[data_diff[data_diff._diff].index]
print(data_diff.shape)
different = significantly_different[significantly_different._count>2].sort_values(by='winner')
print(np.round(float(different.shape[0]) / data_diff.shape[0] * 100, 3),'%')
vals = different[['my_acc','base_acc']].sum().values
print(vals, vals[0]>vals[1])
print(different.shape)

(177, 1)
(9.0399999999999991, '%')
(array([1200, 1169]), True)
(16, 10)


In [20]:
different

_q_type  sum_bottom_up  count_bottom_up  _count  \
7           are there any       0.520157              0.5     172   
63               have her       0.514602              0.5       8   
86              is anyone       0.515345              0.5      95   
144               to walk       0.510516              0.5       4   
24            could these       0.468853              0.5       3   
70   how many animals are       0.467302              0.5      86   
71     how many birds are       0.468498              0.5      86   
72        how many bushes       0.462417              0.5      87   
76       how many pillows       0.466238              0.5     117   
117              name the       0.456662              0.5       3   
167        what is in the       0.470097              0.5     356   
168        what is on the       0.462054              0.5     584   
187             where has       0.466865              0.5       3   
191           where might       0.464655              0.5       3   
198                   who       0.467151              0.5     137   
199                who is       0.464034              0.5     391   

     over_median     winner  my_acc  base_acc  above_median     _diff  
7           True  bottom_up     148       149          True -0.040314  
63          True  bottom_up       4         5         False -0.029203  
86          True  bottom_up      88        89          True -0.030691  
144        False  bottom_up       3         3         False -0.021033  
24         False   top_down       2         2         False  0.062294  
70          True   top_down      33        20          True  0.065396  
71          True   top_down      37        26          True  0.063004  
72          True   top_down      44        36          True  0.075166  
76          True   top_down      84        84          True  0.067525  
117         True   top_down       1         0         False  0.086675  
167         True   top_down     213       203          True  0.059805  
168         True   top_down     310       308          True  0.075892  
187        False   top_down       1         1         False  0.066271  
191         True   top_down       0         0         False  0.070690  
198         True   top_down      56        61          True  0.065699  
199         True   top_down     176       182          True  0.071933